In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!wget -q https://github.com/karoldvl/ESC-50/archive/master.zip
!unzip -q master.zip
# !ls ESC-50-master/audio

In [ ]:
!unzip -q "/content/gdrive/MyDrive/tomo_data/Final_Training Dataset.zip"
!unzip -q "./Final_Training Dataset/train.zip"
!rm 'train/.DS_Store'
!unzip -q /content/gdrive/MyDrive/tomo_data/public_test.zip
!unzip -q /content/gdrive/MyDrive/tomo_data/private_test.zip
!ls

 ESC-50-master		   gdrive     master.zip     public_test   train
'Final_Training Dataset'   __MACOSX   private_test   sample_data


In [ ]:
!pip install pytorch-lightning==1.3.4 --quiet
!pip install torchaudio --quiet

     |████████████████████████████████| 808kB 4.2MB/s 
     |████████████████████████████████| 235kB 13.0MB/s 
     |████████████████████████████████| 122kB 14.0MB/s 
     |████████████████████████████████| 10.6MB 14.0MB/s 
     |████████████████████████████████| 645kB 33.1MB/s 
     |████████████████████████████████| 829kB 35.7MB/s 
     |████████████████████████████████| 1.3MB 36.1MB/s 
     |████████████████████████████████| 296kB 36.5MB/s 
     |████████████████████████████████| 143kB 32.9MB/s 
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
     |████████████████████████████████| 1.9MB 4.0MB/s 


In [ ]:
import librosa
import pandas as pd
import numpy as np
import pickle as pkl 
import time
from functools import reduce
from itertools import cycle
import re
from pathlib import Path
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchvision
import torchvision.models as models
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix

ESC50_DATA_PATH = Path("ESC-50-master/audio")
esc50_df = pd.read_csv("ESC-50-master/meta/esc50.csv")
esc50_others = { 
 'breathing', 'brushing_teeth', 'can_opening', 'car_horn', 'cat', 'clapping',
 'coughing', 'crackling_fire', 'crickets', 'crying_baby', 'door_wood_creaks',
 'door_wood_knock', 'drinking_sipping', 'engine', 'fireworks', 'footsteps',
 'hand_saw', 'helicopter', 'insects', 'keyboard_typing', 'mouse_click',
 'pouring_water', 'rain', 'sneezing', 'snoring', 'toilet_flush', 'vacuum_cleaner',
 'washing_machine', 'water_drops', 'wind'}
esc50_df = esc50_df[esc50_df.category.isin(esc50_others.union({'dog', 'glass_breaking', 'laughing'}))].reset_index()
esc50_df['Label'] = 5
esc50_df.loc[esc50_df.category == 'dog', 'Label'] = 0
esc50_df.loc[esc50_df.category == 'glass_breaking', 'Label'] = 4
esc50_df.loc[esc50_df.category == 'laughing', 'Label'] = 9
ESC50_RANDOM_SEED = 1234
esc50_df, esc50_df_valid, _, _ = train_test_split(esc50_df, esc50_df, test_size=0.025, stratify=esc50_df.category, random_state=ESC50_RANDOM_SEED)
_esc50_df_049, _esc50_df_5 = esc50_df[esc50_df['Label'].isin({0,4,9})], esc50_df[esc50_df['Label']==5]
_esc50_df_5_more, _esc50_df_5, _, _ = train_test_split(_esc50_df_5, _esc50_df_5, test_size=15*_esc50_df_5.category.nunique(), stratify=_esc50_df_5.category, random_state=ESC50_RANDOM_SEED)
esc50_df = pd.concat([_esc50_df_049, _esc50_df_5], axis=0).reset_index()

COLUMNS = "Barking,Howling,Crying,COSmoke,GlassBreaking,Other,Doorbell,Bird,Music_Instrument,Laugh_Shout_Scream"
MODEL_SAVE_PATH = Path("/content/gdrive/MyDrive/tomo_model/")
TRAIN_DATA_PATH = Path("train")
TEST_DATA_PATH = Path("public_test")
PRIVATE_TEST_DATA_PATH = Path("private_test")
train_df = pd.read_csv("./Final_Training Dataset/meta_train.csv")
with open(MODEL_SAVE_PATH / '20210710' / 'label_smoothing.pkl', 'rb') as handle:
    train_data_label_smoothing = pkl.load(handle)
    train_data_label_smoothing = {row['Filename']+".wav": train_data_label_smoothing[i] for i, row in enumerate(train_df.to_dict('records'))}
train_data = {row['Filename']+".wav": row['Label'] for row in train_df.to_dict('records')}
train_data['train_00103.wav'] = 7
id2label = {i: x for i, x in enumerate(COLUMNS.split(","))}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
print(len(list(TRAIN_DATA_PATH.iterdir())))
print(len(list(TEST_DATA_PATH.iterdir())))
train_df.groupby(["Label", "Remark"]).size()

1600
10000


Label  Remark            
0      Barking               200
1      Howling               200
2      Crying                200
3      COSmoke               200
4      GlassBreaking         199
5      Blender                10
       Cat                    30
       Dishes                 60
       Electrics              10
       Other                   1
       Vacuum                 90
6      Doorbell              100
7      Bird                  100
8      Music_Instrument      100
9      Laugh_Shout_Scream    100
dtype: int64

In [ ]:
# ## Label smoothing loss 計算實驗

# tmp_file_name = "train_00103.wav"
# print(train_data[tmp_file_name])
# print(train_data_label_smoothing[tmp_file_name])
# tmp_target = torch.LongTensor([train_data[tmp_file_name]])
# tmp_target_vec = torch.FloatTensor(train_data_label_smoothing[tmp_file_name])

# tmp_outputs = torch.FloatTensor(train_data_label_smoothing[tmp_file_name]).unsqueeze(0)
# tmp_ce_loss = -tmp_target_vec.unsqueeze(0)*tmp_outputs
# tmp_ce_loss = tmp_ce_loss.sum() / tmp_outputs.shape[0]
# print(F.nll_loss(tmp_target_vec.unsqueeze(0), tmp_target))
# print(tmp_ce_loss)

7
[0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.9500000000000001, 0.0, 0.0]
tensor(-0.9500)
tensor(-0.9050)


In [ ]:
SAMPLING_RATE = 8000
num_channels = 3
window_sizes = [25, 50, 100]
hop_sizes = [10, 25, 50]
eps = 1e-6
limits = ((-2, 2), (0.9, 1.2))

def extract_feature(file_path):
    clip, sr = librosa.load(file_path, sr=SAMPLING_RATE)
    specs = []
    for i in range(num_channels):
        window_length = int(round(window_sizes[i]*SAMPLING_RATE/1000))
        hop_length = int(round(hop_sizes[i]*SAMPLING_RATE/1000))            
        clip = torch.Tensor(clip)
        spec = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLING_RATE, n_fft=4410, win_length=window_length, hop_length=hop_length, n_mels=128)(clip)
        spec = spec.numpy()
        spec = np.log(spec+eps)
        spec = np.asarray(torchvision.transforms.Resize((128, 250))(Image.fromarray(spec)))
        specs.append(spec)
    new_entry = {}
    new_entry["audio"] = clip.numpy()
    new_entry["values"] = np.array(specs)
    return new_entry

def aug_audio(audio):
    pitch_shift = np.random.randint(limits[0][0], limits[0][1] + 1)
    time_stretch = np.random.random() * (limits[1][1] - limits[1][0]) + limits[1][0]
    clip = librosa.effects.time_stretch(librosa.effects.pitch_shift(audio, SAMPLING_RATE, pitch_shift), time_stretch)
    specs = []
    for i in range(num_channels):
        window_length = int(round(window_sizes[i]*SAMPLING_RATE/1000))
        hop_length = int(round(hop_sizes[i]*SAMPLING_RATE/1000))            
        clip = torch.Tensor(clił)
        spec = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLING_RATE, n_fft=4410, win_length=window_length, hop_length=hop_length, n_mels=128)(clip)
        spec = spec.numpy()
        spec = np.log(spec+eps)
        spec = np.asarray(torchvision.transforms.Resize((128, 250))(Image.fromarray(spec)))
        specs.append(spec)
    return np.array(specs)

def extract_features(filenames, dir_path, has_target=True, esc50_target=None):
    values = []
    for i, filename in enumerate(filenames):
        if (i+1) % 300 == 0: print(f"{i+1} / {len(filenames)}")
        new_entry = extract_feature(dir_path / filename)
        if has_target:
            new_entry["target"] = train_data[filename] if esc50_target is None else esc50_target
            if esc50_target is None:
                new_entry["target_vec"] = train_data_label_smoothing[filename]
            else:
                new_entry["target_vec"] = [0.0 for _ in range(10)]
                new_entry["target_vec"][esc50_target] = 1.0
        values.append(new_entry)
    return values

In [ ]:
## ESC50
# value_counts
# 9      40
# 5      450
# 4      40
# 0      40

class CycleData():
    def __init__(self, data):
        self.data = data[:]
        self.data = random.sample(self.data, len(self.data))
        self.cyc = cycle(range(len(self.data)))
    
    def get(self):
        idx = next(self.cyc)
        # if idx == 0: self.data = random.sample(self.data, len(self.data))
        return self.data[idx]

esc50_data = dict()
esc50_data_valid = []
for i in [0, 4, 5, 9]:
    print(i)
    esc50_data[i] = CycleData(data=extract_features(list(esc50_df[esc50_df['Label']==i]['filename']), ESC50_DATA_PATH, has_target=True, esc50_target=i))
    esc50_data_valid += extract_features(list(esc50_df_valid[esc50_df_valid['Label']==i]['filename']), ESC50_DATA_PATH, has_target=True, esc50_target=i)

0
4
5
300 / 450
9


In [ ]:
class TomoDataset(Dataset):
    def __init__(self, data, if_balance=False, if_esc50=False, has_target=True):
        self.data = data
        self.length = 250
        self.if_balance = if_balance
        if self.if_balance:
            assert has_target
            target_list = [x['target'] for x in self.data]
            self.class_count = pd.Series(target_list).value_counts().min()
            data_dict = {i: [] for i in range(10)}
            for i, t in enumerate(target_list):
                data_dict[t].append(i)
            for k, v in data_dict.items():
                data_dict[k] = cycle(v)
            self.data_dict = data_dict
        self.has_target = has_target
        self.if_esc50 = if_esc50

    def __len__(self):
        if self.if_balance: 
            return self.class_count * 10
        else:
            return len(self.data)
  
    def __getitem__(self, idx):
        if self.if_balance:
            cls = idx % 10
            idx = next(self.data_dict[cls])
        entry = self.data[idx]
        if self.has_target and self.if_esc50:
            if (entry["target"] in {0, 4, 9} and random.random() < 0.15) or (entry["target"] == 5 and random.random() < 0.4):
                entry = esc50_data[entry["target"]].get()
        values = entry["values"]
        values = torch.Tensor(values.reshape(-1, 128, self.length))
        if self.has_target:
            target = torch.LongTensor([entry["target"]])
            target_vec = torch.FloatTensor(entry["target_vec"])
        else:
            target = torch.LongTensor([-1])
            target_vec = None
        return (values, target, target_vec)

In [ ]:
tmp = extract_features([p.name for p in TRAIN_DATA_PATH.iterdir()][:30], TRAIN_DATA_PATH)
d = TomoDataset(tmp, if_balance=True)
d[4]

(tensor([[[ -5.9427,  -5.0749,  -6.5514,  ...,  -5.5418,  -7.8762,  -8.4243],
          [ -5.1340,  -4.8974,  -6.8065,  ...,  -5.3216,  -7.4580,  -8.0023],
          [ -4.7000,  -4.5260,  -5.9834,  ...,  -5.2242,  -6.8296,  -7.4132],
          ...,
          [ -9.6773,  -5.0002,   3.1101,  ...,  -6.7008,  -7.1380,  -7.6287],
          [-10.8934,  -6.3340,   1.5180,  ...,  -6.7104,  -7.2572,  -8.0624],
          [-11.6693,  -8.4694,  -1.5846,  ...,  -9.5864, -10.2192,  -9.5415]],
 
         [[ -4.4698,  -4.4720,  -5.3616,  ...,  -6.0736,  -7.2000,  -7.1805],
          [ -5.7969,  -4.2027,  -4.0773,  ...,  -4.9718,  -6.6029,  -8.2190],
          [ -4.0580,  -3.5104,  -3.2234,  ...,  -4.5250,  -6.6617,  -9.8180],
          ...,
          [ -8.3721,  -1.6122,   2.8947,  ...,  -6.2859,  -6.4997,  -6.8402],
          [ -9.2171,  -3.3669,   0.9073,  ...,  -6.1151,  -6.4268,  -6.9236],
          [ -9.5197,  -5.5518,  -2.0456,  ..., -11.2725, -10.5420,  -8.0145]],
 
         [[ -3.1864,  -3.333

In [ ]:
BATCH_SIZE = 32
LEARNING_RATE = 0.0002
VAL_SPLIT = .2

RANDOM_SEED= 1234
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
pl.trainer.seed_everything(seed=RANDOM_SEED)

class TomoDataModule(pl.LightningDataModule):

    def __init__(self, batch_size=BATCH_SIZE, val_split=VAL_SPLIT, if_train_all=False):
        super().__init__()
        self.batch_size = batch_size
        self.val_split = val_split
        self.if_train_all = if_train_all
        self._filenames = [p.name for p in sorted(list(TRAIN_DATA_PATH.iterdir())) if p.name.endswith('.wav')]
        self._data = extract_features(self._filenames, TRAIN_DATA_PATH)
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
        self.kf_split = kf.split(X=self._data, y=[x['target'] for x in self._data])
        self.kf_count = 0
        self.update_new_dataset_fold()
    
    def update_new_dataset_fold(self):
        assert self.kf_count < 5
        train_indices, valid_indices = next(self.kf_split)
        self.kf_count += 1
        _train_data, _valid_data = [x for i, x in enumerate(self._data) if self.if_train_all or i in train_indices], [x for i, x in enumerate(self._data) if i in valid_indices]
        self.train_dataset = TomoDataset(data=_train_data, if_balance=True, if_esc50=True)
        self.val_dataset = TomoDataset(data=_valid_data+esc50_data_valid)

    def train_dataloader(self, batch_size=None, shuffle=True):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size if batch_size is None else batch_size, shuffle=shuffle, num_workers=2,
                          pin_memory=self.trainer.on_gpu)

    def val_dataloader(self, batch_size=None, shuffle=False):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size if batch_size is None else batch_size, shuffle=shuffle, num_workers=2,
                          pin_memory=self.trainer.on_gpu)

Global seed set to 1234


In [ ]:
# tmp_data = next(iter(dm.val_dataloader()))
# tmp_input, tmp_target = tmp_data[0], tmp_data[1]
# tmp_outputs = model.forward(tmp_input).to('cpu')
# print(tmp_outputs.squeeze().shape, tmp_target.shape)
# F.nll_loss(tmp_outputs, tmp_target.squeeze(1))

In [ ]:
class TomoModel(pl.LightningModule):
    def __init__(self, learning_rate=LEARNING_RATE, pretrained=True, if_label_smoothing=False):
        super(TomoModel, self).__init__()
        self.tik = time.time()
        self.learning_rate = learning_rate
        self.if_label_smoothing = if_label_smoothing
        num_classes = 10
        self.model = models.densenet161(pretrained=pretrained)
        self.model.classifier = nn.Sequential(
            nn.Linear(2208, num_classes),
            nn.LogSoftmax(dim=1)
        )
        # self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        output = self.model(x)
        return output

    def loss(self, outputs, targets):
        return F.nll_loss(outputs, targets)

    def loss_vec(self, outputs, target_vecs):
        ce_loss = -target_vecs*outputs
        ce_loss = ce_loss.sum() / outputs.shape[0]
        return ce_loss

    def training_step(self, batch, batch_idx):
        inputs, target, target_vec = batch
        outputs = self.forward(inputs)
        if self.if_label_smoothing:
            loss = self.loss_vec(outputs, target_vec)
        else:
            loss = self.loss(outputs, target.squeeze(1))
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, target, target_vec = batch
        outputs = self.forward(inputs)
        if self.if_label_smoothing:
            loss = self.loss_vec(outputs, target_vec)
        else:
            loss = self.loss(outputs, target.squeeze(1))
        prob, pred = torch.max(torch.exp(outputs), 1)
        acc = (pred == target.squeeze(1)).sum().item() / len(pred)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return acc
    
    def validation_epoch_end(self, validation_step_outputs):
        total_acc, cnt = 0, 0
        for acc in validation_step_outputs:
            total_acc += acc
            cnt += 1
        # print(f"    =>   Validation Average Accuracy: {total_acc*100/cnt:.1f} %                      |  epoch time = {(time.time()-self.tik)/60:.1f} min\n")
        print(f"    =>   epoch time = {(time.time()-self.tik)/60:.1f} min\n")
        self.tik = time.time()
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, verbose=True)
        # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,5,7,9], gamma=0.4, verbose=True)
        # return [optimizer], [scheduler]
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [ ]:
def get_val_result(model, dataloader, if_table = False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    n_step = len(dataloader)
    val_result = {'pred': list(), 'prob': list(), 'target': list()}
    val_result_table = []
    hit, cnt = 0, 0
    for i, (inputs, targets, _) in enumerate(dataloader):
        with torch.no_grad():
            out = model.to(device).forward(inputs.to(device))
            if if_table:
                out_prob = torch.exp(out).to('cpu')
                val_result_table.extend(out_prob.tolist())
            prob, pred = torch.max(torch.exp(out).to('cpu'), 1)
            val_result['pred'].extend(pred.tolist())
            val_result['prob'].extend(prob.tolist())
            val_result['target'].extend(targets.squeeze(1).tolist())
            acc = (pred == targets.squeeze(1)).sum().item()
        hit += acc
        cnt += len(pred)
    val_result = pd.DataFrame(val_result)
    print(f"=============================================================        Validation Accuracy: {hit/cnt*100:.1f} %")
    if if_table:
        return (val_result, pd.DataFrame(val_result_table, columns=COLUMNS.split(",")))
    else:
        return val_result

In [ ]:
dm = TomoDataModule(if_train_all=False)

300 / 1600
600 / 1600
900 / 1600
1200 / 1600
1500 / 1600


In [ ]:
TRAIN = True
NUM_EPOCHS = 70
MODEL_SAVE_NAME = 'tomo_model_weights'
KFOLD = True

model = TomoModel(pretrained=True)

if TRAIN:
    logger = TensorBoardLogger('tb_logs', name='my_tomo_model')
    early_stop_callback = EarlyStopping(
        min_delta=0.000,
        patience=6,
        verbose=True,
        monitor='val_acc',
        mode='max'
    )
    checkpoint_callback = ModelCheckpoint(
        dirpath=MODEL_SAVE_PATH,
        filename=MODEL_SAVE_NAME,
        save_top_k=1,
        verbose=True,
        monitor='val_acc',
        mode='max'
    )
    if KFOLD:
        kfold_val_result = []
        for i in range(5):
            model = TomoModel(pretrained=True)
            early_stop_callback = EarlyStopping(
                min_delta=0.000,
                patience=6,
                verbose=True,
                monitor='val_acc',
                mode='max'
            )
            checkpoint_callback = ModelCheckpoint(
                dirpath=MODEL_SAVE_PATH,
                filename=MODEL_SAVE_NAME+f"_fold{i+1}",
                save_top_k=1,
                verbose=True,
                monitor='val_acc',
                mode='max'
            )
            trainer = pl.Trainer(max_epochs=NUM_EPOCHS, 
                                logger=logger, 
                                progress_bar_refresh_rate=20, 
                                gpus=-1 if torch.cuda.is_available() else 0, 
                                auto_lr_find=False,
                                callbacks=[early_stop_callback, checkpoint_callback]
                                 )
            trainer.fit(model, dm)
            # trainer.save_checkpoint(MODEL_SAVE_PATH / (MODEL_SAVE_NAME+f"_fold{i+1}.ckpt"))
            model = TomoModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
            model.eval()
            torch.save(model.state_dict(), MODEL_SAVE_PATH / ('pytorch_'+Path(trainer.checkpoint_callback.best_model_path).stem+'.pkl'))
            kfold_val_result.append(get_val_result(model, dm.val_dataloader()))
            if i < 4: dm.update_new_dataset_fold()
else:
    print(f"Reuse model: {str(MODEL_SAVE_PATH / MODEL_SAVE_NAME)+'.ckpt'}")
    model = TomoModel.load_from_checkpoint(str(MODEL_SAVE_PATH / MODEL_SAVE_NAME)+'.ckpt')

model.eval()
assert model.training is False

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /content/gdrive/MyDrive/tomo_model exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | DenseNet | 26.5 M
-----------------------------------
26.5 M    Trainable params
0         Non-trainable params
26.5 M    Total params
105.976   Total estimated model params size (MB)


Global seed set to 1234


    =>   epoch time = 0.0 min



Metric val_acc improved. New best score: 0.737
Epoch 0, global step 25: val_acc reached 0.73667 (best 0.73667), saving model to "/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold1.ckpt" as top 1


    =>   epoch time = 0.3 min



...

    =>   epoch time = 0.3 min



Epoch 7, global step 207: val_acc was not in top 1


    =>   epoch time = 0.3 min



Monitored metric val_acc did not improve in the last 6 records. Best score: 0.843. Signaling Trainer to stop.
Epoch 8, global step 233: val_acc was not in top 1


    =>   epoch time = 0.3 min


=============================================================        Validation Accuracy: 84.3 %


In [ ]:
# # Start tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir tb_logs

In [ ]:
all_train_dataset = TomoDataset(data=dm._data)
model_save_name_prefix = "tomo_model_weights_7"
all_model_paths = [p for p in (MODEL_SAVE_PATH/"").iterdir() if p.name.startswith(model_save_name_prefix)]
all_train_result = []
all_train_result_table = []
for i in range(len(all_model_paths)):
    # model = TomoModel.load_from_checkpoint(str(MODEL_SAVE_PATH / (model_save_name_prefix+f"{i+1}"))+'.ckpt')
    model = TomoModel.load_from_checkpoint(all_model_paths[i])
    model.eval()
    print(all_model_paths[i])
    a, b = get_val_result(model, DataLoader(all_train_dataset, batch_size=16, shuffle=False), if_table=True)
    all_train_result.append(a)
    all_train_result_table.append(b)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth



/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold1.ckpt


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


=============================================================        Validation Accuracy: 88.6 %
/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold2.ckpt
=============================================================        Validation Accuracy: 89.3 %
/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold3.ckpt
=============================================================        Validation Accuracy: 88.0 %
/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold4.ckpt
=============================================================        Validation Accuracy: 88.9 %
/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold5.ckpt
=============================================================        Validation Accuracy: 88.0 %
/content/gdrive/MyDrive/tomo_model/tomo_model_weights_7_fold6.ckpt
=============================================================        Validation Accuracy: 88.0 %


In [ ]:
print(id2label)
for i in range(len(all_train_result)):
    df = all_train_result[i]
    # true label being i-th class and predicted label being j-th class
    print()
    print(confusion_matrix(y_pred=df.pred, y_true=df.target))

{0: 'Barking', 1: 'Howling', 2: 'Crying', 3: 'COSmoke', 4: 'GlassBreaking', 5: 'Other', 6: 'Doorbell', 7: 'Bird', 8: 'Music_Instrument', 9: 'Laugh_Shout_Scream'}

[[185   4   3   0   1   3   0   2   0   1]
 [  6 167  16   0   0   2   0   0   0   9]
 [  7  13 160   0   2   8   0   2   0   8]
 [  0   0   0 191   0   3   4   2   0   0]
 [  0   0   0   0 199   0   0   0   0   0]
 [  0   0   1   0  13 166   1   2   3  15]
 [  1   0   0   1   1   6  82   0   7   2]
 [  0   0   3   0   0   2   0  94   0   2]
 [  0   0   0   0   0   1   4   0  94   1]
 [  1   0   1   0   3   5   0   3   0  87]]

[[182   2   4   0   3   5   0   1   1   1]
 [  4 156  38   0   0   1   0   0   0   1]
 [ 14  10 173   0   0   2   0   0   1   0]
 [  0   0   0 192   0   2   5   0   1   0]
 [  0   0   0   0 197   2   0   0   0   0]
 [  0   1   5   1   9 179   1   0   1   4]
 [  1   0   0   0   0   2  89   0   7   1]
 [  2   0   2   1   1   1   0  94   0   0]
 [  0   0   0   0   0   1   4   0  95   0]
 [  3   2   5   0 

In [ ]:
avg_pred = (reduce(lambda a, b: a+b, all_train_result_table) / len(all_train_result_table)).apply(lambda row: list(row).index(max(row)), axis=1)
confusion_matrix(y_pred=avg_pred, y_true=all_train_result[0].target)

array([[188,   3,   3,   0,   0,   0,   0,   2,   0,   3],
       [  4, 175,  17,   0,   0,   1,   0,   0,   0,   3],
       [  6,  11, 174,   0,   0,   7,   0,   1,   0,   1],
       [  0,   0,   0, 190,   0,   3,   6,   1,   0,   0],
       [  0,   0,   0,   0, 199,   0,   0,   0,   0,   0],
       [  0,   1,   2,   0,   4, 180,   3,   1,   4,   6],
       [  0,   0,   0,   0,   0,   0,  96,   0,   3,   1],
       [  1,   0,   1,   0,   0,   0,   0,  99,   0,   0],
       [  0,   0,   0,   0,   0,   1,   1,   0,  97,   1],
       [  2,   1,   1,   0,   1,   4,   0,   1,   0,  90]])

In [ ]:
## make table for label smoothing
SMOOTHING = 0.1

train_avg_pred_tbl = (reduce(lambda a, b: a+b, all_train_result_table) / len(all_train_result_table))
result = []
for i in range(len(all_train_dataset)):
    trg = all_train_dataset[i][1].item()
    tmp = [0.0 for _ in range(10)]
    row = train_avg_pred_tbl.iloc[i, ]
    cnt = row[row>=0.08]
    if len(cnt) == 1 and row.argmax() == trg:
        tmp[trg] = 1.0
    else:
        tmp[trg] = 1.0-SMOOTHING
        for j in cnt.index.map(label2id).tolist():
            tmp[j] += SMOOTHING / len(cnt)
    result.append(tmp)

with open(MODEL_SAVE_PATH / 'label_smoothing.pkl', 'wb') as handle:
    pkl.dump(result, handle)

In [ ]:
# with open(MODEL_SAVE_PATH / 'label_smoothing.pkl', 'rb') as handle:
#     train_data_label_smoothing = pkl.load(handle)
# len(train_data_label_smoothing)

1600

In [ ]:
df = all_train_result[4]
sel = (df.target==3) & (df.pred != df.target)
print(df.loc[sel, :])
print(pd.Series(dm._filenames)[sel])
print(train_df.loc[(train_df.Filename+".wav").isin(pd.Series(dm._filenames)[sel]), :])

      pred      prob  target
559      6  0.619998       3
613      6  0.651236       3
735      7  0.930683       3
839      6  0.745434       3
879      5  0.856097       3
1042     6  0.989940       3
1229     6  0.921998       3
1254     7  0.508576       3
1273     6  0.974936       3
1474     6  0.580822       3
1517     7  0.865530       3
559     train_00777.wav
613     train_00731.wav
735     train_00682.wav
839     train_00751.wav
879     train_00785.wav
1042    train_00623.wav
1229    train_00618.wav
1254    train_00750.wav
1273    train_00795.wav
1474    train_00635.wav
1517    train_00770.wav
dtype: object
        Filename  Label   Remark
617  train_00618      3  COSmoke
622  train_00623      3  COSmoke
634  train_00635      3  COSmoke
681  train_00682      3  COSmoke
730  train_00731      3  COSmoke
749  train_00750      3  COSmoke
750  train_00751      3  COSmoke
769  train_00770      3  COSmoke
776  train_00777      3  COSmoke
784  train_00785      3  COSmoke
794  train_

# Inference

In [ ]:
test_filenames = sorted([p.name for p in TEST_DATA_PATH.iterdir()])
private_test_filenames = sorted([p.name for p in PRIVATE_TEST_DATA_PATH.iterdir()])
test_filenames_list = [
    (test_filenames, TEST_DATA_PATH), 
    (private_test_filenames[0:5000], PRIVATE_TEST_DATA_PATH), 
    (private_test_filenames[5000:10000], PRIVATE_TEST_DATA_PATH),
    (private_test_filenames[10000:15000], PRIVATE_TEST_DATA_PATH),
    (private_test_filenames[15000:20000], PRIVATE_TEST_DATA_PATH)
]

In [ ]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_save_name_prefix = "tomo_model_weights"
all_model_paths = [p for p in (MODEL_SAVE_PATH/"20210602").iterdir() if p.name.startswith(model_save_name_prefix)]
test_result_table_list = []
for (_test_filenames, _data_path) in test_filenames_list:
    _test_data = extract_features(_test_filenames, _data_path, has_target=False)
    test_dataset = TomoDataset(data=_test_data, has_target=False)
    test_result_table = []
    for i in range(len(all_model_paths)):
        print(all_model_paths[i].name)
        model = TomoModel.load_from_checkpoint(all_model_paths[i])
        model.eval()
        test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        n_step = len(test_dataloader)
        result = []
        for j, (inputs, _) in enumerate(test_dataloader):
            # if (j+1) % int(n_step/4) == 0:
            #     print(f"{(j+1) // int(n_step/4)} / 4")
            with torch.no_grad():
                out = model.to(device).forward(inputs.to(device))
                out_prob = torch.exp(out).to('cpu')
                result.extend(out_prob.tolist())
        test_result_table.append(pd.DataFrame(result, columns="Barking,Howling,Crying,COSmoke,GlassBreaking,Other".split(",")))
    test_result_table_list.append(test_result_table)
    del _test_data

300 / 10000
600 / 10000
900 / 10000
1200 / 10000
1500 / 10000
1800 / 10000
2100 / 10000
2400 / 10000
2700 / 10000
3000 / 10000
3300 / 10000
3600 / 10000
3900 / 10000
4200 / 10000
4500 / 10000
4800 / 10000
5100 / 10000
5400 / 10000
5700 / 10000
6000 / 10000
6300 / 10000
6600 / 10000
6900 / 10000
7200 / 10000
7500 / 10000
7800 / 10000
8100 / 10000
8400 / 10000
8700 / 10000
9000 / 10000
9300 / 10000
9600 / 10000
9900 / 10000
tomo_model_weights_fold1_2.ckpt
tomo_model_weights_fold1_1.ckpt
tomo_model_weights_fold2_1.ckpt
tomo_model_weights_fold2_2.ckpt
tomo_model_weights_fold3_1.ckpt
tomo_model_weights_fold3_2.ckpt
tomo_model_weights_fold4_1.ckpt
tomo_model_weights_fold4_2.ckpt
tomo_model_weights_fold5_1.ckpt
tomo_model_weights_fold5_2.ckpt
300 / 5000
600 / 5000
900 / 5000
1200 / 5000
1500 / 5000
1800 / 5000
2100 / 5000
2400 / 5000
2700 / 5000
3000 / 5000
3300 / 5000
3600 / 5000
3900 / 5000
4200 / 5000
4500 / 5000
4800 / 5000
tomo_model_weights_fold1_2.ckpt
tomo_model_weights_fold1_1.ckpt
t

In [ ]:
result_df_list = []
for (i, _test_result_table) in enumerate(test_result_table_list):
    result_df = pd.DataFrame((reduce(lambda a, b: a+b, _test_result_table) / len(_test_result_table)), columns="Barking,Howling,Crying,COSmoke,GlassBreaking,Other".split(",")).applymap('{:,.6f}'.format)
    result_df['Filename'] = [re.sub(r"\.wav$", '', x) for x in test_filenames_list[i][0]]
    result_df = result_df.sort_values(by='Filename').loc[:, ["Filename"]+"Barking,Howling,Crying,COSmoke,GlassBreaking,Other".split(",")].reset_index(drop=True)
    result_df_list.append(result_df)
result_df = pd.concat(result_df_list, axis=0).reset_index(drop=True)

In [ ]:
samp_subm = pd.read_csv("/content/gdrive/MyDrive/tomo_data/sample_submission.csv")
# not given private test set
# result_df = pd.concat([result_df, samp_subm.loc[~samp_subm.Filename.isin(set(result_df.Filename)), :]], axis=0).reset_index(drop=True)
# given private test set
sort_idx = dict(zip(samp_subm['Filename'], range(len(samp_subm))))
result_df = result_df.sort_values(by='Filename', key=lambda x: x.map(sort_idx), ascending=True)
assert result_df.shape == samp_subm.shape
result_df

,Filename,Barking,Howling,Crying,COSmoke,GlassBreaking,Other
0,public_00001,0.000859,0.002050,0.000572,0.000696,0.000438,0.995385
1,public_00002,0.000379,0.000265,0.000461,0.000210,0.000613,0.998073
2,public_00003,0.247694,0.007957,0.025308,0.590576,0.048049,0.080417
3,public_00004,0.000358,0.000154,0.000188,0.000185,0.001441,0.997674
4,public_00005,0.397496,0.019735,0.341938,0.013330,0.212307,0.015194
...,...,...,...,...,...,...,...
29995,private_19996,0.001298,0.002247,0.001478,0.855812,0.028761,0.110403
29996,private_19997,0.151344,0.005100,0.027973,0.008897,0.004560,0.802126
29997,private_19998,0.000247,0.000346,0.000533,0.000284,0.000209,0.998380
29998,private_19999,0.236174,0.163003,0.310929,0.079598,0.026294,0.184003


In [ ]:
result_df.to_csv('my_submission.csv', index=False)
result_df.to_csv(MODEL_SAVE_PATH/'my_submission.csv', index=False)